기상자료개방포털 사이트에서는 API 서비스를 운영하고 있다  
이를 이용해서 현재 기상 데이터를 가져와 다음날 평균기온을 추론하는 코드를 작성해보려한다

In [16]:
import requests
from urllib import parse
import tensorflow as tf
import keras
from keras.models import load_model 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

api를 통해 데이터를 불러온다  
참고 글 :  https://jvvp.tistory.com/1035

In [23]:
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
key = 'cT4JNsYj6lxarKAfoCWMZH4YY4mzzyc7kS3%2BNb1CYJVFh614%2FO6qMi%2FLZTSmCVHXkY7%2B3JsSdOClQ8RtBE9APQ%3D%3D'



param = f'?{parse.quote_plus("ServiceKey")}={key}&' + parse.urlencode({
    parse.quote_plus('numOfRows') : '10',
    parse.quote_plus('pageNo') : '1',
    parse.quote_plus('dataCd') : 'ASOS',
    parse.quote_plus('dateCd') : 'DAY',
    parse.quote_plus('startDt') : '20210101',
    parse.quote_plus('endDt') : '20210102',
    parse.quote_plus('stnIds') : '108'})

response = requests.get(url+param)
print(response.text)

soup = BeautifulSoup(response.content, 'lxml')

<?xml version="1.0" encoding="UTF-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><stnId>108</stnId><stnNm>서울</stnNm><tm>2021-01-01</tm><avgTa>-4.2</avgTa><minTa>-9.8</minTa><minTaHrmt>511</minTaHrmt><maxTa>1.6</maxTa><maxTaHrmt>1447</maxTaHrmt><mi10MaxRnHrmt></mi10MaxRnHrmt><hr1MaxRn></hr1MaxRn><hr1MaxRnHrmt></hr1MaxRnHrmt><sumRn></sumRn><maxInsWs>7.7</maxInsWs><maxInsWsWd>270</maxInsWsWd><maxInsWsHrmt>1530</maxInsWsHrmt><maxWs>4.1</maxWs><maxWsWd>270</maxWsWd><maxWsHrmt>1532</maxWsHrmt><avgWs>2.0</avgWs><hr24SumRws>1723</hr24SumRws><maxWd>270</maxWd><avgTd>-10.2</avgTd><minRhm>50</minRhm><minRhmHrmt>1140</minRhmHrmt><avgRhm>64.0</avgRhm><avgPv>2.9</avgPv><avgPa>1014.9</avgPa><maxPs>1028.1</maxPs><maxPsHrmt>249</maxPsHrmt><minPs>1023.9</minPs><minPsHrmt>1354</minPsHrmt><avgPs>1026.0</avgPs><ssDur>9.6</ssDur><sumSsHr>6.5</sumSsHr><hr1MaxIcsrHrmt>1100</hr1MaxIcsrHrmt><hr1MaxIcsr>1.74</hr1MaxIcsr><

이렇게 api로 호출한 데이터를 데이터프레임 형태로 정리해보자  
아래 태그명은 공공데이터포털사이트에서 제공해주는 활용가이드를 사용하여 정리하였다  

In [25]:
col_raw = 'stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRnHrmt,hr1MaxRn,hr1MaxRnHrmt,sumRn,maxInsWs,maxInsWsWd,maxInsWsHrmt,maxWs,maxWsWd,maxWsHrmt,avgWs,hr24SumRws,maxWd,avgTd,minRhm,minRhmHrmt,avgRhm,avgPv,avgPa,maxPs,maxPsHrmt,minPs,minPsHrmt,avgPs,ssDur,sumSsHr,hr1MaxIcsrHrmt,hr1MaxIcsr,sumGsr,ddMefs,ddMefsHrmt,ddMes,ddMesHrmt,sumDpthFhsc,avgTca,avgLmac,avgTs,minTg,avgCm5Te,avgCm10Te,avgCm20Te,avgCm30Te,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur'
col_raw = col_raw.lower()
col=col_raw.split(',')

In [26]:
data = []
cnt=0
for c in col:
    data.append([i.text for i in soup.select('item '+c)])

학습데이터와 같은 방식으로 전처리하기 위해 동일한 컬럼명으로 바꿔준다.  
'강수 계속시간(hr)','10분 최다 강수량(mm)' 이 2가지 값은 csv로 다운받았을 때는 포함되어 있었지만 api로 불러왔을 때는 포함되지 않았다  

In [27]:
col_name = ['지점', '지점명', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)', '최고기온(°C)',
            '최고기온 시각(hhmi)', '10분 최다강수량 시각(hhmi)','1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)',
            '일강수량(mm)', '최대 순간 풍속(m/s)','최대 순간 풍속 풍향(16방위)', '최대 순간풍속 시각(hhmi)', '최대 풍속(m/s)',
            '최대 풍속 풍향(16방위)','최대 풍속 시각(hhmi)', '평균 풍속(m/s)', '풍정합(100m)', '최다풍향(16방위)',
            '평균 이슬점온도(°C)', '최소 상대습도(%)', '최소 상대습도 시각(hhmi)', '평균 상대습도(%)','평균 증기압(hPa)',
            '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최고 해면기압 시각(hhmi)','최저 해면기압(hPa)', '최저 해면기압 시각(hhmi)',
            '평균 해면기압(hPa)', '가조시간(hr)','합계 일조시간(hr)', '1시간 최다일사 시각(hhmi)', '1시간 최다일사량(MJ/m2)',
            '합계 일사량(MJ/m2)','일 최심신적설(cm)', '일 최심신적설 시각(hhmi)', '일 최심적설(cm)', '일 최심적설 시각(hhmi)',
            '합계 3시간 신적설(cm)', '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)','최저 초상온도(°C)',
            '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)','평균 20cm 지중온도(°C)', '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)',
            '1.0m 지중온도(°C)', '1.5m 지중온도(°C)', '3.0m 지중온도(°C)', '5.0m 지중온도(°C)',
            '합계 대형증발량(mm)', '합계 소형증발량(mm)', '9-9강수(mm)', '기사', '안개 계속시간(hr)']

In [29]:
DF = pd.DataFrame(data).T
DF.columns = col_name
DF

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),10분 최다강수량 시각(hhmi),1시간 최다강수량(mm),...,0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),기사,안개 계속시간(hr)
0,108,서울,2021-01-01,-4.2,-9.8,511,1.6,1447,,,...,2.9,6.8,9.7,15.9,17.5,1.1,1.6,,,
1,108,서울,2021-01-02,-5.0,-8.4,805,-1.4,1346,,,...,2.6,6.6,9.6,15.8,17.5,1.4,2.0,,,
